# Logic for creating the delivery fee

#### Author : Rahul Bhoyar

#### Date : 25/01/2024

## Rules for calculating a delivery fee

* If the cart value is less than 10€, a small order surcharge is added to the delivery price. The surcharge is the difference between the cart value and 10€. For example if the cart value is 8.90€, the surcharge will be 1.10€.
* A delivery fee for the first 1000 meters (=1km) is 2€. If the delivery distance is longer than that, 1€ is added for every additional 500 meters that the courier needs to travel before reaching the destination. Even if the distance would be shorter than 500 meters, the minimum fee is always 1€.
  * Example 1: If the delivery distance is 1499 meters, the delivery fee is: 2€ base fee + 1€ for the additional 500 m => 3€
  * Example 2: If the delivery distance is 1500 meters, the delivery fee is: 2€ base fee + 1€ for the additional 500 m => 3€
  * Example 3: If the delivery distance is 1501 meters, the delivery fee is: 2€ base fee + 1€ for the first 500 m + 1€ for the second 500 m => 4€
* If the number of items is five or more, an additional 50 cent surcharge is added for each item above and including the fifth item. An extra "bulk" fee applies for more than 12 items of 1,20€
  * Example 1: If the number of items is 4, no extra surcharge
  * Example 2: If the number of items is 5, 50 cents surcharge is added
  * Example 3: If the number of items is 10, 3€ surcharge (6 x 50 cents) is added
  * Example 4: If the number of items is 13, 5,70€ surcharge is added ((9 * 50 cents) + 1,20€)
  * Example 5: If the number of items is 14, 6,20€ surcharge is added ((10 * 50 cents) + 1,20€)
* The delivery fee can __never__ be more than 15€, including possible surcharges.
* The delivery is free (0€) when the cart value is equal or more than 200€.
* During the Friday rush, 3 - 7 PM, the delivery fee (the total fee including possible surcharges) will be multiplied by 1.2x. However, the fee still cannot be more than the max (15€). Considering timezone, for simplicity, **use UTC as a timezone in backend solutions** (so Friday rush is 3 - 7 PM UTC). **In frontend solutions, use the timezone of the browser** (so Friday rush is 3 - 7 PM in the timezone of the browser).

In [ ]:
def calculate_surcharge_for_lower_cart_value_order(cart_value):
    minimum_cart_value = 1000
    surcharge = max(0, minimum_cart_value - cart_value)
    return surcharge

def calculate_surcharge_for_large_orders(num_items):
    item_surcharge_threshold = 4
    bulk_fee_threshold = 12
    item_surcharge = max(0, (num_items - item_surcharge_threshold) * 50)
    bulk_fee = 120 if num_items > bulk_fee_threshold else 0

    total_surcharge = item_surcharge + bulk_fee
    return total_surcharge


def calculate_delivery_fee_in_cents_for_distance(distance):
    base_fee_cents = 200
    additional_fee_per_500m_cents = 100

    if distance <= 1000:
        return base_fee_cents

    # Calculate additional fees for each 500 meters beyond the first kilometer
    additional_distance = max(0, distance - 1000)  # Distance beyond the first kilometer

    if additional_distance % 500 == 0:
        additional_fee_cents = (additional_distance // 500) * additional_fee_per_500m_cents
    else:
        additional_fee_cents = ((additional_distance // 500) + 1) * additional_fee_per_500m_cents

    # Add additional fees to the base fee
    delivery_fee_cents = base_fee_cents + additional_fee_cents

    return delivery_fee_cents


def friday_rush_multiplier(delivery_fee,time):
    from datetime import datetime
    FRIDAY_RUSH_START = 15  # 3 PM in 24-hour format
    FRIDAY_RUSH_END = 19  # 7 PM in 24-hour format
    FRIDAY_RUSH_MULTIPLIER = 1.2

    order_time = datetime.strptime(time, "%Y-%m-%dT%H:%M:%SZ")
    if order_time.weekday() == 4 and FRIDAY_RUSH_START <= order_time.hour < FRIDAY_RUSH_END:
        delivery_fee = delivery_fee * FRIDAY_RUSH_MULTIPLIER
        return delivery_fee
    else:
        return delivery_fee


def delivery_fee_calculation(cart_value, delivery_distance, number_of_items, time):
    if cart_value >= 20000:
        return 0
    # Step 1
    surcharge_for_lower_cart_value_order = calculate_surcharge_for_lower_cart_value_order(cart_value)
    print("surcharge_for_lower_cart_value_order :",surcharge_for_lower_cart_value_order)

    # Step 2
    delivery_fee_in_cents_for_distance = calculate_delivery_fee_in_cents_for_distance(delivery_distance)
    print("delivery_fee_in_cents_for_distance :",delivery_fee_in_cents_for_distance)

    # Step 3
    surcharge_for_large_orders = calculate_surcharge_for_large_orders(number_of_items)
    print("surcharge_for_large_orders :",surcharge_for_large_orders)

    total_delivery_fee = surcharge_for_lower_cart_value_order + delivery_fee_in_cents_for_distance + surcharge_for_large_orders

    #Step 4 : Friday rush case
    total_delivery_fee = int(friday_rush_multiplier(total_delivery_fee,time))

    if total_delivery_fee > 1500:
        return 1500

    return total_delivery_fee



In [ ]:
#test case 1 : Normal case

number_of_items = 4
distance = 2235
cart_value = 790
time = "2024-01-15T13:00:00Z"

total_delivery_fee = delivery_fee_calculation(cart_value, distance, number_of_items, time)
print("Final Deivery fee :",total_delivery_fee)

surcharge_for_lower_cart_value_order : 210
delivery_fee_in_cents_for_distance : 500
surcharge_for_large_orders : 0
Final Deivery fee : 710


In [ ]:
#test case 2 : If Cart value is greater than 20000

number_of_items = 4
distance = 2235
cart_value = 22790
time = "2024-01-15T13:00:00Z"

total_delivery_fee = delivery_fee_calculation(cart_value, distance, number_of_items, time)
print("Final Deivery fee :",total_delivery_fee)

Final Deivery fee : 0


In [ ]:
#test case 3 :

number_of_items = 3
distance = 700
cart_value = 1300
time = "2024-09-19T13:00:00Z"

total_delivery_fee = delivery_fee_calculation(cart_value, distance, number_of_items, time)
print("Final Deivery fee :",total_delivery_fee)

surcharge_for_lower_cart_value_order : 0
delivery_fee_in_cents_for_distance : 200
surcharge_for_large_orders : 0
Final Deivery fee : 200


In [ ]:
#test case 4 : Friday Rush 1

number_of_items = 13
distance = 3100
cart_value = 890
time = "2024-01-19T16:00:00Z"

total_delivery_fee = delivery_fee_calculation(cart_value, distance, number_of_items, time)
print("Final Deivery fee :",total_delivery_fee)

surcharge_for_lower_cart_value_order : 110
delivery_fee_in_cents_for_distance : 700
surcharge_for_large_orders : 570
Final Deivery fee : 1500


In [ ]:
#test case 5 : Friday Rush 2

friday_rush_time = "2024-01-19T16:00:00Z"  # 4 PM UTC
cart_value = 1500
delivery_distance = 100
number_of_items = 4
time = friday_rush_time


total_delivery_fee = delivery_fee_calculation(cart_value, distance, number_of_items, time)
print("Final Deivery fee :",total_delivery_fee)

surcharge_for_lower_cart_value_order : 0
delivery_fee_in_cents_for_distance : 700
surcharge_for_large_orders : 0
Final Deivery fee : 840


In [ ]:
#test case 1

number_of_items = 4
distance = 2235
cart_value = 790
time = "2024-01-15T13:00:00Z"

total_delivery_fee = delivery_fee_calculation(cart_value, distance, number_of_items, time)
print("Final Deivery fee :",total_delivery_fee)

surcharge_for_lower_cart_value_order : 210
delivery_fee_in_cents_for_distance : 500
surcharge_for_large_orders : 0
Final Deivery fee : 710
